In [8]:
'''
This is a script that will read in a few different csv files, as per the 
Udacity data analysis course instructions.
'''

import csv

# function to read in csv files, file's name as the arg
def read_csv(filename):
    # open the data file as a dictionary
    with open(filename, "rb") as f:
        reader = csv.DictReader(f)
        return list(reader)

# test the function on some csv files
#print(read_csv('enrollments.csv')[0])
enrollments = read_csv('enrollments.csv')
daily_engagement = read_csv('daily_engagement.csv')
project_submissions = read_csv('project_submissions.csv')

In [10]:
enrollments[0]

{'account_key': '448',
 'cancel_date': '2015-01-14',
 'days_to_cancel': '65',
 'is_canceled': 'True',
 'is_udacity': 'True',
 'join_date': '2014-11-10',
 'status': 'canceled'}

In [11]:
daily_engagement[0]

{'acct': '0',
 'lessons_completed': '0.0',
 'num_courses_visited': '1.0',
 'projects_completed': '0.0',
 'total_minutes_visited': '11.6793745',
 'utc_date': '2015-01-09'}

In [15]:
project_submissions[0]

{'account_key': '256',
 'assigned_rating': 'UNGRADED',
 'completion_date': '2015-01-16',
 'creation_date': '2015-01-14',
 'lesson_key': '3176718735',
 'processing_state': 'EVALUATED'}

# 5 Questions About These Datasets:
### 1)  Can we correlate the amount of total projects submitted with the duration of enrollment?
### 2)  What is the relationship between the number of submitted projects and the amount of daily engagement?
### 3)  What is the relationship between the number of projects completed and whether the student has a cancelled account?
### 4)  What is the relationship between the projects in "Ungraded" current enrollment status?
### 5)  What is the relationship between the daily engagement for a student and the number of projects passed?

In [16]:
enrollment_num_rows = len(enrollments)
print enrollment_num_rows

1640


In [30]:
def get_unique_accounts(dictionary):
    unique_keys = set()
    for key in dictionary:
        unique_keys.add(key['acct'])
    len(unique_keys)
    return len(unique_keys)

1302

In [29]:
def get_unique_keys(dictionary):
    unique_keys = set()
    for key in dictionary:
        unique_keys.add(key['account_key'])
    len(unique_keys)
    return len(unique_keys)
